In [45]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment, Edge, Node
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from sample.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests

In [46]:
directory = "/projects/0/einf914/data/"
exp = get_exp((10,104,104),directory)

begin = 2020-12-17 08:39:00 
  end = 2020-12-17 08:39:00


In [47]:
exp.load_compressed_skel()

In [48]:
plt.close('all')
exp.plot([0],[[node.label for node in exp.nodes if node.degree(0)>=3]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
graph = exp.nx_graph[0]
graph.get_edge_data(3593,3544)['width']

29.13572518957827

In [16]:
Node(3593,exp).neighbours(0)

[Node(3229), Node(3620), Node(3544)]

In [28]:

def get_width_pixel(edge,index,im,pivot,before,after,t,size = 20,width_factor = 60,averaging_size = 100,threshold_averaging = 10):
    imtab=im
#     print(imtab.shape)
#     print(int(max(0,pivot[0]-averaging_size)),int(pivot[0]+averaging_size))
    threshold = np.mean(imtab[int(max(0,pivot[0]-averaging_size)):int(pivot[0]+averaging_size),int(max(0,pivot[1]-averaging_size)):int(pivot[1]+averaging_size)]-threshold_averaging)
    orientation=np.array(before)-np.array(after)
    perpendicular = [1,-orientation[0]/orientation[1]] if orientation[1]!=0 else [0,1]
    perpendicular_norm=np.array(perpendicular)/np.sqrt(perpendicular[0]**2+perpendicular[1]**2)
    point1=np.around(np.array(pivot)+width_factor*perpendicular_norm)
    point2=np.around(np.array(pivot)-width_factor*perpendicular_norm)
    point1=point1.astype(int)
    point2=point2.astype(int)
    p = profile_line(imtab, point1, point2,mode='constant')
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(p)
    derivative = [p[i+1]-p[i] for i in range(len(p)-1)]
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot([np.mean(derivative[5*i:5*i+5]) for i in range(len(derivative)//5)])
    problem=False
    arg = len(p)//2
    if p[arg]>threshold:
        arg = np.argmin(p)
#     we_plot=randrange(1000)
    while  p[arg]<=threshold:
        if arg<=0:
#             we_plot=50
            problem=True
            break
        arg-=1
    begin = arg
    arg = len(p)//2
    if p[arg]>threshold:
        arg = np.argmin(p)
    while  p[arg]<=threshold:
        if arg>=len(p)-1:
#             we_plot=50
            problem=True
            break
        arg+=1
    end = arg
#     print(end-begin,threshold)
    return(np.linalg.norm(point1-point2)*(end-begin)/len(p))

def get_width_edge(edge,resolution,t,local=False, threshold_averaging = 10):
    pixel_conversion_factor = 1.725
    pixel_list = edge.pixel_list(t)
    pixels = []
    indexes = []
    source_images = []
    poss = []
    widths={}
    if len(pixel_list)>3*resolution:
        for i in range(0,len(pixel_list)//resolution):
            index = i*resolution
            indexes.append(index)
            pixel = pixel_list[index]
            pixels.append(pixel)
            source_img,pos = get_source_image(edge.experiment,pixel,t,local)
            source_images.append(source_img)
            poss.append(pos)
    else:
        indexes = [0,len(pixel_list)//2,len(pixel_list)-1]
        for index in indexes:
            pixel = pixel_list[index]
            pixels.append(pixel)
            source_img,pos = get_source_image(edge.experiment,pixel,t,local)
            source_images.append(source_img)
            poss.append(pos)
#     print(indexes)
    for i, index in enumerate(indexes[1:-1]):
        source_img = source_images[i+1]
        pivot = poss[i+1]
        _,before = get_source_image(edge.experiment,pixels[i],t,local,pivot)
        _,after = get_source_image(edge.experiment,pixels[i+2],t,local,pivot)
#         plot_t_tp1([0,1,2],[],{0 : pivot,1 : before, 2 : after},None,source_img,source_img)
        width = get_width_pixel(edge,index,source_img,pivot,before,after,t,threshold_averaging = threshold_averaging)
#         print(width*pixel_conversion_factor)
        widths[pixel_list[index]]=width*pixel_conversion_factor
    edge.experiment.nx_graph[t].get_edge_data(edge.begin.label,edge.end.label)['width'] = widths
    return(widths)      

In [29]:
get_width_edge(Edge(Node(3593,exp),Node(3229,exp),exp),50,0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{(18237, 25209): 42.70320414263475,
 (18189, 25198): 44.226431389369324,
 (18140, 25186): 42.70320414263475,
 (18094, 25174): 40.84201533366425,
 (18047, 25160): 40.824398205571676,
 (18000, 25152): 44.361807528721755,
 (17950, 25144): 42.655584162232465,
 (17902, 25135): 40.862460291967295,
 (17853, 25125): 40.864939360072434,
 (17805, 25115): 40.94936079574316,
 (17757, 25108): 42.655584162232465,
 (17711, 25099): 42.429813937726735,
 (17662, 25090): 40.824398205571676,
 (17615, 25076): 39.32114070861803,
 (17567, 25062): 37.42236502177404}

In [18]:
# plt.close('all')
Node(3229,exp).show_source_image(0,0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
def get_width_hypha(hypha,t):
    nodes, edges = hypha.get_nodes_within(t)
    widths = [edge.width(t) for edge in edges]
    lengths = [len(edge.pixel_list(t)) for edge in edges]
    weighted = [widths[i]*lengths[i] for i in range(len(widths))]
    return(np.sum(weighted)/np.sum(lengths))

def get_rh_bas(exp,criter):
    select_hyph = get_hyph_infos(exp)
    max_speeds = []
    total_growths = []
    lengths = []
    branch_frequ = []
    hyph_l = []
    RH=[]
    BAS=[]
    widths=[]
    for hyph in exp.hyphaes:
        speeds = [c[2] for c in select_hyph[hyph]]
        ts = [c[0] for c in select_hyph[hyph]]
        tp1s = [c[1] for c in select_hyph[hyph]]
        if len(speeds)>0:
            length = hyph.get_length_um(hyph.ts[-1])
            nodes = hyph.get_nodes_within(hyph.ts[-1])[0]
            max_speed = np.max(speeds)
            total_growth = np.sum([speed * get_time(exp,ts[i],tp1s[i]) for i,speed in enumerate(speeds)])
            if criter(max_speed,length):
                RH.append(hyph)
            else:
                BAS.append(hyph)
            lengths.append(length)
            max_speeds.append(max_speed)
            total_growths.append(total_growth)
            branch_frequ.append((len(nodes)-1)/(length+1))
            hyph_l.append(hyph)
            widths.append(get_width_hypha(hyph,hyph.ts[-1]))
        else:
            BAS.append(hyph)            
    return(RH, BAS, max_speeds, total_growths, widths, lengths, branch_frequ,select_hyph)

In [5]:
widths = [(hyph,get_width_hypha(hyph,hyph.ts[-1])) for hyph in exp.hyphaes if len(hyph.ts)>0]

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [15]:
# width_real =[c for c in widths if c[1]<160]
width_real = widths

widths_fin = [c[1] for c in width_real]
lengths = [c[0].get_length_um(c[0].ts[-1]) for c in width_real]

In [16]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(widths_fin,lengths)
ax.set_xlabel(r'width ($\mu m $)')
ax.set_ylabel(r'length ($\mu m $)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'length ($\\mu m $)')

In [8]:
RH, BAS, max_speeds, total_growths, widths_sp, lengths, branch_frequ,select_hyph = get_rh_bas(get_exp((40,0,37),directory),criter)

begin = 2020-11-19 20:21:00 
  end = 2020-11-25 18:06:00


/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [13]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(widths_sp,max_speeds)
ax.set_xlabel(r'width ($\mu m $)')
ax.set_ylabel(r'max speed ($\mu m.s^{-1}$)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'max speed ($\\mu m.s^{-1}$)')

In [12]:

widths_rh = [width for width in widths_rh if width<160]
widths_bas = [width for width in widths_bas if width<160]
fig = plt.figure()
bins = np.linspace(0, 160, 30)
ax = fig.add_subplot(111)
ax.hist(widths_rh,bins,alpha=0.5,label='rh',density=True)
ax.hist(widths_bas,bins,alpha=0.5,label='bas',density=True)
ax.set_xlabel(r'width ($\mu m $)')
plt.legend(loc='upper right')
# ax.set_ylabel(r'length ($\mu m $)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [137]:
BAS = [hyph for hyph in BAS if len(hyph.ts)>1]


In [11]:
widths_rh = [get_width_hypha(hyph,hyph.ts[-1]) for hyph in RH]
widths_bas = [get_width_hypha(hyph,hyph.ts[-1]) for hyph in BAS if len(hyph.ts)>0]

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [129]:

widths_rh = [width for width in widths_rh if width<160]
widths_bas = [width for width in widths_bas if width<160]
fig = plt.figure()
bins = np.linspace(0, 160, 30)
ax = fig.add_subplot(111)
ax.hist(widths_rh,bins,alpha=0.5,label='rh',density=True)
ax.hist(widths_bas,bins,alpha=0.5,label='bas',density=True)
ax.set_xlabel(r'width ($\mu m $)')
plt.legend(loc='upper right')
# ax.set_ylabel(r'length ($\mu m $)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [181]:
hyph = choice(BAS)
[get_width_hypha(hyph,t) for t in hyph.ts]

[30.151660321266935,
 32.21005792690662,
 34.23827774386727,
 30.141020949279866,
 29.27306187180139,
 27.539231147236237]

In [182]:
plt.close('all')
t=0
hyph.end.show_source_image(hyph.ts[t],hyph.ts[t])
hyph.root.show_source_image(hyph.ts[t],hyph.ts[t])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [173]:
exp.plot([hyph.ts[0]],[[hyph.end.label,hyph.root.label]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(hyph.ts,[get_width_hypha(hyph,t) for t in hyph.ts])
ax.set_xlabel('timestep')
ax.set_ylabel(r'width ($\mu m $)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'width ($\\mu m $)')

In [61]:
times=[]
widths_t=[]
for rh in RH:
    for t in rh.ts:
        width = get_width_hypha(rh,t)
        if width<80:
            times.append(get_time(exp,rh.ts[0],t))
            widths_t.append(width)

In [62]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(times,widths_t)
ax.set_xlabel('timestep')
ax.set_ylabel(r'width ($\mu m $)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'width ($\\mu m $)')

In [64]:
slope, intercept, r_value, p_value, std_err = stats.linregress(times,speeds)

In [68]:
speeds= widths_t
zipped_list = zip(times,speeds)
sorted_zip = sorted(zipped_list)
times_sort,speed_sort = zip(*sorted_zip)
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time since appearane of hypha ($hour$)')
ax.set_ylabel('width ($\mu m)$')
xlim = 140
ax.set_xlim((0,xlim))
ax.set_ylim((0,100))

x= range(0,xlim)

N=100
ax.scatter(times_sort,speed_sort,color='yellow',label='observed width')
ax.plot(uniform_filter1d(times_sort, size=N),uniform_filter1d(speed_sort, size=N),color='green',label = 'moving average')
# ax.set_title(r'$r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p_value = {round(p_value**2,max(2,-int(np.log10(p_value**2))+2))}, slope = {round(slope,4)}')
x= range(0,xlim+1)
legend_str = r'linear regression : $r^2=$'+f'{round(r_value**2,max(2,-int(np.log10(r_value**2))+2))}, p-value = {round(p_value,max(2,-int(np.log10(p_value))+2))}, slope = {round(slope,4)}'
line, = ax.plot(x,x*slope+intercept,color='red',label = legend_str)
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
hyph.end.show_source_image(hyph.ts[1],hyph.ts[1])
hyph.root.show_source_image(hyph.ts[1],hyph.ts[1])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
hyph.end.ts(),hyph.root.ts()

([35, 36, 37], [36, 37])

In [43]:
plt.close('all')
exp.plot(hyph.ts,[[hyph.end.label,hyph.root.label]]*len(hyph.ts))

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
roots=[]
for t in hyph.end.ts():
    #             print(t,tip)
    if hyph.end.degree(t) == 1:
        root, edges, nodes = hyph.get_edges(t, 200)
        roots.append(root)
occurence_count = Counter(roots)
occurence_count

Counter({Node(4819): 2, Node(5274): 1})

In [115]:
clean_exp_with_hyphaes(exp)

0.0
0.03638348189921776
0.07276696379843552
0.10915044569765327
0.14553392759687103
0.18191740949608878
0.21830089139530653
0.2546843732945243
0.29106785519374206
0.3274513370929598
0.36383481899217757
0.4002183008913953
0.43660178279061307
0.4729852646898308
0.5093687465890486
0.5457522284882663
0.5821357103874841
0.6185191922867018
0.6549026741859196
0.6912861560851373
0.7276696379843551
0.7640531198835728
0.8004366017827906
0.8368200836820083
0.8732035655812261
0.9095870474804438
0.9459705293796616
0.9823540112788793
Detected problems during hyphae detection, 118 hyphaes have inconsistent root over time
removing  350  nodes out of 12710 because they appear only in one timestep and are not within an identified hypha
9.163170799027931
13.776128455643143
14.218210236496239
19.64906114270876
19.279700676541903
19.94349411283939
38.75439473285086
27.982052980273824
61.68227112654536
58.776285142604
95.40498457929924
79.82866056635241
41.46663930007891
67.6333382064431
36.036074977466434


NameError: name 'reconnect_degree_2' is not defined